In [264]:
import numpy as np
import pandas as pd
import re
import nltk.classify.util
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.classify import NaiveBayesClassifier

%matplotlib inline


In [265]:
df = pd.read_csv('rating_df.csv', encoding = 'latin1')

In [266]:
type(df)

pandas.core.frame.DataFrame

In [267]:
df = df.drop('Unnamed: 0', axis = 1)

In [268]:
df.isnull().sum() #There are nan values in the dataset

R1    0
R2    0
R3    0
R4    0
R5    1
R6    1
dtype: int64

In [269]:
x = df.unstack()

In [270]:
x = x.dropna()

In [271]:
x[:5]

R1  0    (' 5 ', "  This game is awesome. Its addicting...
    1    (' 4 ', '  Absolutely love all of the geometry...
    2    (' 2 ', '  Game is completely different than t...
    3    (' 4 ', "  I REALLY like to save this little s...
    4    (' 4 ', "  CANDY CRUSH is the very first game ...
dtype: object

In [272]:
x[0][3]

'5'

In [273]:
len(x)

1798

In [274]:
stars = []
for i in np.arange(0,len(x)):
    star = x[i][3]
    stars.append(star)
    
print(stars)

['5', '4', '2', '4', '4', '5', '5', '1', '5', '5', '5', '4', '5', '5', '1', '4', '5', '3', '3', '4', '4', '5', '5', '5', '3', '5', '1', '3', '4', '2', '4', '1', '5', '4', '3', '3', '4', '5', '5', '1', '5', '2', '5', '5', '5', '1', '5', '5', '4', '5', '4', '1', '5', '5', '3', '4', '3', '3', '1', '3', '5', '4', '1', '5', '4', '3', '4', '5', '2', '4', '5', '1', '4', '4', '3', '5', '1', '5', '3', '3', '5', '3', '5', '3', '5', '1', '4', '3', '5', '4', '5', '5', '3', '5', '5', '3', '5', '5', '5', '1', '5', '2', '4', '3', '2', '3', '1', '4', '2', '5', '5', '2', '5', '1', '5', '5', '4', '2', '5', '5', '4', '1', '5', '2', '1', '4', '2', '1', '5', '5', '4', '5', '5', '5', '5', '5', '2', '2', '4', '3', '1', '3', '5', '4', '4', '4', '5', '5', '2', '4', '3', '3', '5', '5', '2', '4', '4', '1', '4', '4', '5', '4', '1', '5', '1', '5', '5', '2', '1', '4', '4', '5', '5', '1', '5', '3', '2', '1', '5', '3', '5', '2', '3', '5', '3', '4', '4', '5', '3', '3', '5', '4', '5', '2', '3', '5', '2', '3', '1', '5',

/Users/cherylto/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/multi.py:819: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return self._engine.get_value(s, k)


In [275]:
len(stars)

1798

In [276]:
stars_numeric = pd.to_numeric(stars)
stars_numeric

array([5, 4, 2, ..., 3, 2, 5])

In [277]:
comments = []
for i in np.arange(0, len(x)):
    comment = x[i][11:]
    comments.append(comment)
    
print(comments)

['This game is awesome. Its addicting and I love it. I\'ve played games on Google play. Out of all of them this is the best one. I\'m not sure why anyone is criticising the game and why they have to pay for some features. I\'m sure that if they created a game they would want to make money on it. Or even need to keep the upkeep on it and performance or ad levels. I don\'t think people would want to use their own money from their regular job to keep up such a massive undertaking. They would be homeless if they did. Therefore wouldn\'t even get to play games such as this. You also have to remember that the only right you have in the U.S. is to spend money, and fun cost money. But n-e-ways great game guys.  ")', "Absolutely love all of the geometry dash games! Colors and graphics are awesome, game play is challenging but fun, and the music is perfect! Only thing I can complain about is that I wish all of the levels from these free versions were in the full version. I own the full version a

/Users/cherylto/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/multi.py:819: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return self._engine.get_value(s, k)


In [278]:
df2 = pd.DataFrame({'star_rating': stars_numeric, 'review_comments': comments})

In [279]:
df2.shape

(1798, 2)

In [280]:
df2.head()

,review_comments,star_rating
0,This game is awesome. Its addicting and I love...,5
1,Absolutely love all of the geometry dash games...,4
2,Game is completely different than the ad I fol...,2
3,I REALLY like to save this little sausage life...,4
4,CANDY CRUSH is the very first game that I got ...,4


Separating into positive and negative comments

In [281]:
pos = df2[df2['star_rating'] >3]

In [282]:
neg = df2[df2['star_rating'] <=3]

In [283]:
print(len(pos), len(neg))

997 801


In [284]:
print(len(pos) + len(neg))

1798


In [285]:
pos.head()

,review_comments,star_rating
0,This game is awesome. Its addicting and I love...,5
1,Absolutely love all of the geometry dash games...,4
3,I REALLY like to save this little sausage life...,4
4,CANDY CRUSH is the very first game that I got ...,4
5,A lot of fun and a challenge to your mind! Th...,5


In [286]:
neg.head()

,review_comments,star_rating
2,Game is completely different than the ad I fol...,2
7,Rating this a 1 star until my issue is resolve...,1
14,First im not spending money on upgrades if i c...,1
17,It is set up so that its brightness does not m...,3
18,"Cool game, great material design! Everything i...",3


In [287]:
pos.to_csv('pos.csv', encoding = 'utf-8')

In [288]:
neg.to_csv('neg.csv', encoding = 'utf-8')

In [289]:
pos_comments = pos['review_comments']

In [290]:
def clean_text(text):
    bytes(text, 'utf-8').decode("unicode_escape")
    return text

In [291]:
clean_comments_pos = pos_comments.apply(clean_text)

In [292]:
clean_comments_pos.head()

0    This game is awesome. Its addicting and I love...
1    Absolutely love all of the geometry dash games...
3    I REALLY like to save this little sausage life...
4    CANDY CRUSH is the very first game that I got ...
5    A lot of fun and a challenge to your mind!  Th...
Name: review_comments, dtype: object

In [293]:
neg_comments = neg['review_comments']

In [294]:
clean_comments_neg = neg_comments.apply(clean_text)

In [295]:
neg_comments.head()

2     Game is completely different than the ad I fol...
7     Rating this a 1 star until my issue is resolve...
14    First im not spending money on upgrades if i c...
17    It is set up so that its brightness does not m...
18    Cool game, great material design! Everything i...
Name: review_comments, dtype: object

<b> Starting Sentiment Analysis

In [296]:
def create_word_features(words):
    useful_words = [word for word in words if word not in stopwords.words('english')]
    lower_words = [word.lower() for word in useful_words]
    my_dict = dict([(word, True) for word in lower_words])
    
    return my_dict

In [297]:
#z = ['Cat', 'Ted', 'Tom']

In [298]:
#create_word_features(z)

<b> Negative

In [299]:
neg_tokens=[]
for string in neg_comments:
    word = word_tokenize(string)
    neg_tokens.append((create_word_features(word), 'negative'))
    
print(len(neg_tokens))
print(neg_tokens[:5])   

801
[({'game': True, 'completely': True, 'different': True, 'ad': True, 'i': True, 'followed': True, 'shows': True, '.': True, 'recent': True, 'update': True, 'reformabts': True, 'screen': True, 'progress': True, 'looks': True, 'lot': True, 'like': True, 'board': True, ',': True, 'also': True, 'adds': True, 'bunch': True, 'moving': True, 'effects': True, 'unnecessary': True, 'actually': True, 'distracting': True, 'constant': True, 'prompts': True, 'buy': True, 'coinsnot': True, 'sure': True, 'app': True, "'": True, ')': True}, 'negative'), ({'rating': True, '1': True, 'star': True, 'issue': True, 'resolved': True, '.': True, 'we': True, 'paid': True, '10.00': True, 'linked': True, 'account': True, 'came': True, 'time': True, 'download': True, 'phone': True, ',': True, 'purchased': True, 'levels': True, 'naver': True, 'so': True, 'basically': True, 'gave': True, 'nothing': True, 'son': True, 'game': True, 'week': True, "'": True, ')': True}, 'negative'), ({'first': True, 'im': True, 'sp

<b> Positive

In [300]:
pos_tokens=[]
for string in pos_comments:
    word = word_tokenize(string)
    pos_tokens.append((create_word_features(word), 'positive'))
    
print(pos_tokens[:5])
print(len(pos_tokens))

[({'this': True, 'game': True, 'awesome': True, '.': True, 'its': True, 'addicting': True, 'i': True, 'love': True, "'ve": True, 'played': True, 'games': True, 'google': True, 'play': True, 'out': True, 'best': True, 'one': True, "'m": True, 'sure': True, 'anyone': True, 'criticising': True, 'pay': True, 'features': True, 'created': True, 'would': True, 'want': True, 'make': True, 'money': True, 'or': True, 'even': True, 'need': True, 'keep': True, 'upkeep': True, 'performance': True, 'ad': True, 'levels': True, "n't": True, 'think': True, 'people': True, 'use': True, 'regular': True, 'job': True, 'massive': True, 'undertaking': True, 'they': True, 'homeless': True, 'therefore': True, 'get': True, 'you': True, 'also': True, 'remember': True, 'right': True, 'u.s.': True, 'spend': True, ',': True, 'fun': True, 'cost': True, 'but': True, 'n-e-ways': True, 'great': True, 'guys.': True, '``': True, ')': True}, 'positive'), ({'absolutely': True, 'love': True, 'geometry': True, 'dash': True, 

<b> Create Training Set

In [301]:
train_set = pos_tokens[:797] + neg_tokens[:639]
test_set = pos_tokens[797:] + neg_tokens[639:]
print(len(train_set), len(test_set))

1436 362


<b> Create the Naive Bayes Classifier

In [302]:
classifier = NaiveBayesClassifier.train(train_set)

In [303]:
accuracy = nltk.classify.util.accuracy(classifier, test_set)
print(accuracy*100)

77.07182320441989


In [304]:
fidget_review1 = """It was good I liked how I got a brand new one that fast"""
print(fidget_review1) #This comment was a 5* review

It was good I liked how I got a brand new one that fast


In [305]:
words = word_tokenize(fidget_review1)
words = create_word_features(words)
classifier.classify(words)

'positive'

In [306]:
fidget_review2 = """I think that you'll did a really good game and you'll should make more games"""
print(fidget_review2) #This comment was a 5* review

I think that you'll did a really good game and you'll should make more games


In [307]:
words = word_tokenize(fidget_review2)
words = create_word_features(words)
classifier.classify(words)

'positive'

In [308]:
fidget_review3 = """Downloaded this for free money on a different game, it sucks so much. And i didnt even get the copleted ad reward, dont download uninstalling"""
print(fidget_review2) #This comment was a 1* review

I think that you'll did a really good game and you'll should make more games


In [309]:
words = word_tokenize(fidget_review3)
words = create_word_features(words)
classifier.classify(words)

'negative'